In [21]:
import os

import numpy as np
import pandas as pd
import cv2 as cv
from matplotlib import pyplot as plt

from tqdm import tqdm
from glob import glob

from pathlib import Path

# custom patched dataset

In [22]:
SCALE_FACTOR = 0.5
PATCH_SIZE = 1024

In [23]:
def make_path_to_img(id: str):
    ans = (
        './../data/train_images/{}.tiff'.format(id),
        './../data/train_masks/{}.npy'.format(id)
    )
    return ans

In [47]:
!rm ./../data/train_masks_patches_1024_overlap/*.png

In [48]:
!rm ./../data/train_images_patches_1024_overlap/*.png

In [31]:
# !mkdir ./../data/train_images_patches_1024_overlap/
# !mkdir ./../data/train_masks_patches_1024_overlap/

In [51]:
def patchify_image(id: str, output_dir_img: str, output_dir_mask: str):
    Path(output_dir_img).mkdir(parents=True, exist_ok=True)
    Path(output_dir_mask).mkdir(parents=True, exist_ok=True)
    
    iname, mname = make_path_to_img(id)
    
    img = cv.imread(iname)
    img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
    
    mask = np.load(mname)
    
    img = cv.resize(img, None, fx=SCALE_FACTOR, fy=SCALE_FACTOR, interpolation=cv.INTER_AREA)
    mask = cv.resize(mask, dsize=img.shape[:2], interpolation=cv.INTER_NEAREST)
    
#     thresh = img.mean(axis=2).astype(np.uint8)
    
#     color = np.array([0, 0, 0])
#     img = np.stack([np.pad(img[:,:,c], PATCH_SIZE // 4, mode='constant', constant_values=color[c]) for c in range(3)], axis=2)

#     mask = np.pad(mask, PATCH_SIZE // 4, mode='constant', constant_values=color[0])
    
#     thresh = np.pad(thresh, PATCH_SIZE // 4, mode='constant', constant_values=255)
#     _, thresh = cv.threshold(thresh, int(thresh.mean()), 255, cv.THRESH_BINARY)
    
#     plt.imshow(thresh, cmap='gray')
#     plt.show()
    
#     plt.imshow(img)
#     plt.show()
    
#     plt.imshow(mask)
#     plt.show()
    
    H, W, _ = img.shape
    
    patch_index = 0
    for dy in np.round(np.linspace(0, H - PATCH_SIZE - 1, 4)).astype(np.int32):
        for dx in np.round(np.linspace(0, W - PATCH_SIZE - 1, 4)).astype(np.int32):
#     for dy in range(PATCH_SIZE // 8, H - PATCH_SIZE, PATCH_SIZE // 5):
#         for dx in range(PATCH_SIZE // 8, W - PATCH_SIZE, PATCH_SIZE // 5):
#             th_slc = thresh[dy : dy + PATCH_SIZE, dx : dx + PATCH_SIZE]
#             th_slc = 1 - th_slc / 255
#             th_sum = th_slc.sum()
#             if th_sum < 1000:
#                 continue
            
            
            patch = img[dy : dy + PATCH_SIZE, dx : dx + PATCH_SIZE]
            maskpatch = mask[dy : dy + PATCH_SIZE, dx : dx + PATCH_SIZE]
            
            cv.imwrite(os.path.join(output_dir_img, '{}_{:05d}.png'.format(id, patch_index)), patch[:, :, ::-1])
            cv.imwrite(os.path.join(output_dir_mask, '{}_{:05d}.png'.format(id, patch_index)), maskpatch)
            
            patch_index += 1
            
#             print(mask.min(), mask.max())
#             0/0

In [52]:
patchify_image(
    '10044', 
    './../data/train_images_patches_1024_overlap/', 
    './../data/train_masks_patches_1024_overlap/'
)

In [53]:
for fname in tqdm(glob('./../data/train_images/*.tiff')):
    bname = os.path.basename(fname)
    id = bname.split('.')[0]
    patchify_image(
        id, 
        './../data/train_images_patches_1024_overlap/', 
        './../data/train_masks_patches_1024_overlap/'
    )    

100%|██████████| 351/351 [09:45<00:00,  1.67s/it]
